# 05.Building a Classification Model with Spark

- 분류(Classification)는 범주 또는 종류로 구분하는 것
- 분류의 종류로는 이진분류와 멀티클래스 분류가 있음.
- 분류는 학습용 데이터에 정답 또는 관심있는 값을 지정하고 모델을 학습시키는 지도학습(supervised learning)임.
- 군집과 차이는 ??

![A simple binary classification problem](Classification_01.jpg "A simple binary classification problem")

![A simple binary classification problem](Classification_02.jpg "A simple binary classification problem")

### 분류의 용도
- 1) 인터넷 사용자가 광고를 클릭할지 여부를 예측
- 2) 사기 거래 여부를 예측
- 3) 부도 여부를 예측
- 4) images, video, sounds의 분류
- 5) 새로운 뉴스 또는 페이지의 분류
- 6) 스팸메일 및 해킹 여부를 판단
- 7) 고객의 등급 분류
- 8) 고객 이탈 예측

### 이번 장의 목표
- 1) SparkML에서 사용할 수 있는 분류모델의 종류을 논의
- 2) raw 데이터로부터 알맞는 특징을 추출하는 방법 알아보기
- 3) 분류모델을 학습하는 방법 알아보기
- 4) 학습된 모델을 이용해서 예측기 만들기
- 5) 예측기의 성능을 측정할 수 있는 표준적인 평가방법을 알아보기
- 6) 여러가지 특성값 추출방법을 이용해서 모델의 성능을 향상법을 알아보기
- 7) 모델 성능에 관련된 튜닝 파라메타이 영향도를 알아보고, 최적의 모델 파라메타를 선택하기 위한 교차분석법을 배워보자.

# 1. Types of classification models

- 1) 선형모델 : 매우 큰 데이터셋에 적용하기 쉬음.
- 2) 나무모델 : 스케일업에 다소 어려움이 있지만, 강력한 비선형 모델을 제공
- 3) 나이브 베이즈모델 : 더 단순하지만, 효과적인 학습과 병렬처리가 쉬음

- 이진분류기 : 선형모델, 나무모델, 나이브베이즈 모델을 SparkML에서 지원
- 멀티클래스 분류기 : 나무모델, 나이브베이즈 모델을 SparkML에서 지원

## 1.1. Linear models

- 입력변수들을 단순한 선형 함수를 사용해서 목표변수( 종속변수 )를 예측하는 모형
- 행렬을 이용한 중회귀모형
![행렬을 이용한 중회귀모형](regress_01.jpg "행렬을 이용한 중회귀모형")
![행렬을 이용한 중회귀모형](regress_02.jpg "행렬을 이용한 중회귀모형")
![행렬을 이용한 중회귀모형](regress_03.jpg "행렬을 이용한 중회귀모형")
![행렬을 이용한 중회귀모형](regress_04.jpg "행렬을 이용한 중회귀모형")

- 로지스틱 회귀모형 
![로지스틱 회귀모형](Classification_03.jpg "로지스틱 회귀모형")
![로지스틱 회귀모형](Classification_04.jpg "로지스틱 회귀모형")
![로지스틱 회귀모형](Classification_05.jpg "로지스틱 회귀모형")
![로지스틱 회귀모형](Classification_06.jpg "로지스틱 회귀모형")

## 1.2. Linear support vector machines

- 확률적인 모델이 아닌, 모델평가를 긍정/부정 인지를 기반하여 분류을 예측함.
- SVM의 손실함수는 hinge loss라고 하면 아래와 같음.
![loss function for SVM](Classification_07.jpg "loss function for SVM")

- Decision functions for logistic regression and linear SVM for binary classification
![Decision functions for logistic regression and linear SVM for binary classification](Classification_08.jpg "Decision functions for logistic regression and linear SVM for binary classification")

## 1.3. The naïve Bayes model

- 주어진 학습용 데이터로부터 각각의 분류의 가장 가능도가 높은 분포개형을 계산하고, 이를 기반으로 분류

![The naïve Bayes model](Classification_09.jpg "The naïve Bayes model")
![The naïve Bayes model](Classification_10.jpg "The naïve Bayes model")
![The naïve Bayes model](Classification_11.jpg "The naïve Bayes model")

## 1.4. Decision trees

- 아주 복잡한 비선형 패턴이나 특성치들의 상호작용을 잡아낼 수 있는 강력하고, 비확률적인 기법
- 집에 있을지, 야외에 놀러나갈지를 결정 방식을 도식화
![Decision trees](Classification_12.jpg "Decision trees")

- 나이브베이즈 모형과 비교해보자. 
![Decision trees](Classification_13.jpg "Decision trees")

# 2. Extracting the right features from your data

- 특성치(독립변수)는 숫자의 vector형식으로 주로 다루어지고, 분류기나 회귀는 지도학습이기 때문에 feature vector와 target value가 필요함.
- Spark MLlib에서는 LabeledPoint 클래스를 지원함.
- case class LabeledPoint(label: Double, features: Vector)

## 2.1. Extracting features from the Kaggle/StumbleUpon evergreen classification dataset

- 데이터셋은 StumbleUpon에서 주어짐.
- 자신들의 웹 컨텐츠 추천 페이지에 있는 웹페이지에서 단기간만 살아있을 것(non-evergreen)과 오래동안 인기(evergreen)가 있을것을 분류하는 문제임.
- http://www.kaggle.com/c/stumbleupon/data
- train.tsv 파일을 다운로드 
- train.tsv의 각각의 필드 설명
![The following table includes field descriptions for train.tsv](Classification_14.jpg "The following table includes field descriptions for train.tsv")



- 헤더가 포함되어 있어서 첫번째 라인을 제거함.

In [ ]:
sed 1d train.tsv > train_noheader.tsv

- Spark shell을 시작하고 학습데이터를 읽어서 한줄씩 탭으로 자름

In [ ]:
bin/spark-shell –-driver-memory 4g

val rawData = sc.textFile("data/train_noheader.tsv")
val records = rawData.map(line => line.split("\t"))
records.first()

- 1번째와 2번째 컬럼은 URL과 페이지ID
- 다음 2개의 컬럼은 문자열 데이터 
- 다음 22개의 컬럼은 숫자 또는 범주형 데이터
- 마지막 컬럼은 목표값으로 1이면 evergreen, 0이면 non-evergreen


- 이진 범주형 데이터에 대해서, 이미 1-of-k encoding 을 가지고 있어서 추가적인 변수 추출은 하지 않음
- 데이터중에서 ? 문자열을  0.0으로 할당함.
- label은 정수형, features은 Array[Double] 임.

In [ ]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors
val data = records.map { r =>
    val trimmed = r.map(_.replaceAll("\"", ""))
    val label = trimmed(r.size - 1).toInt
    val features = trimmed.slice(4, r.size - 1).map(d => if (d =="?") 0.0 else d.toDouble)
    LabeledPoint(label, Vectors.dense(features))
}
data.cache
val numData = data.count

- 나이브베이즈 모델에서는 음수값에 대해서는 에러를 발생하기 때문에 음수를 0.0으로 변환하는 버전을 구현

In [ ]:
val nbData = records.map { r =>
    val trimmed = r.map(_.replaceAll("\"", ""))
    val label = trimmed(r.size - 1).toInt
    val features = trimmed.slice(4, r.size - 1).map(d => if (d =="?") 0.0 else d.toDouble).map(d => if (d < 0) 0.0 else d)
    LabeledPoint(label, Vectors.dense(features))
}
val numNbData = nbData.count

# 3. Training classification models

- logistic regression, SVM, naïve Bayes, and a decision tree 각각을 모두 훈련시킴.
- 최고의 파라메터 세팅을 위해서 평가기법을 사용함.

# 3.1. Training a classification model on the Kaggle/StumbleUpon evergreen classification dataset

In [ ]:
import org.apache.spark.mllib.classification.LogisticRegressionWithSGD
import org.apache.spark.mllib.classification.SVMWithSGD
import org.apache.spark.mllib.classification.NaiveBayes
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.configuration.Algo
import org.apache.spark.mllib.tree.impurity.Entropy
val numIterations = 10
val maxTreeDepth = 5

val lrModel = LogisticRegressionWithSGD.train(data, numIterations)

val svmModel = SVMWithSGD.train(data, numIterations)

val nbModel = NaiveBayes.train(nbData)

val dtModel = DecisionTree.train(data, Algo.Classification, Entropy, maxTreeDepth)

# 4. Using classification models

## 4.1. Generating predictions for the Kaggle/StumbleUpon evergreen classification dataset

In [ ]:
val dataPoint = data.first
val prediction = lrModel.predict(dataPoint.features)
val trueLabel = dataPoint.label

val predictions = lrModel.predict(data.map(lp => lp.features))
predictions.take(5)

# 5. Evaluating the performance of classification models

![accuracy precision recall](Classification_15.jpg "accuracy  precision recall")

- precision와  recall는 역의 관계를 가짐
![precision recall](Classification_16.jpg "precision recall")

- Precision-recall curve
![Precision-recall curve](Classification_20.jpg "Precision-recall curve")

- precision와  recall의 조화평균을 F1 값이라고 함.
![precision recall](Classification_17.jpg "precision recall")

![ROC](Classification_18.jpg "ROC")

![AUC](Classification_19.jpg "AUC")

## 5.1. Accuracy and prediction error

In [ ]:
// 로지스틱 선형분류의 정확도 
val lrTotalCorrect = data.map { point =>
    if (lrModel.predict(point.features) == point.label) 1 else 0
}.sum
val lrAccuracy = lrTotalCorrect / data.count

// SVM의 정확도
val svmTotalCorrect = data.map { point =>
    if (svmModel.predict(point.features) == point.label) 1 else 0
}.sum
val svmAccuracy = svmTotalCorrect / numData

// naïve Bayes 의 정확도
val nbTotalCorrect = nbData.map { point =>
    if (nbModel.predict(point.features) == point.label) 1 else 0
}.sum
val nbAccuracy = nbTotalCorrect / numData

// decision tree의 정확도
val dtTotalCorrect = data.map { point =>
    val score = dtModel.predict(point.features)
    val predicted = if (score > 0.5) 1 else 0
    if (predicted == point.label) 1 else 0
}.sum
val dtAccuracy = dtTotalCorrect / numData


## 5.2. Precision and recall

- sparkMLlib에서는 이진분류기의 PR값과  ROC curves값을 구하는 모듈이 내장됨.

In [ ]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

// 선형분류와 SVN 모델 
val metrics = Seq(lrModel, svmModel).map { model =>
    val scoreAndLabels = data.map { point =>
        (model.predict(point.features), point.label)
    }

    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName, metrics.areaUnderPR, metrics.areaUnderROC)
}

// naïve Bayes 모델
val nbMetrics = Seq(nbModel).map{ model =>
    val scoreAndLabels = nbData.map { point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
                                 
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    ( model.getClass.getSimpleName, metrics.areaUnderPR, metrics.areaUnderROC )
}

// decision tree
val dtMetrics = Seq(dtModel).map{ model =>
    val scoreAndLabels = data.map { point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
                                 
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName, metrics.areaUnderPR, metrics.areaUnderROC)
}

// 화면 출력
val allMetrics = metrics ++ nbMetrics ++ dtMetrics
allMetrics.foreach{ case (m, pr, roc) =>
    println(f"$m, Area under PR: ${pr * 100.0}%2.4f%%, Area under ROC: ${roc * 100.0}%2.4f%%")
}


# 6. Improving model performance and tuning parameters

## 6.1 Feature standardization

- 많은 수의 모델들이 입력 데이터의 분포를 가정하고 있음. 예를 들면, 정규분포
- 그러므로, 입력 데이터의 분포를 알아보는것이 필요
- SparkMLlib에서는 RowMatrix 클래스를 사용해서 feature vectors( 독립변수들, X값들 )을 분포 matrix로 표현함.
- RowMatrix 클래스를 matrix의 컬럼들의 통계량을 계산해주는 유용한 Utility을 제공함.

In [ ]:
import org.apache.spark.mllib.linalg.distributed.RowMatrix
val vectors = data.map(lp => lp.features)
val matrix = new RowMatrix(vectors)
val matrixSummary = matrix.computeColumnSummaryStatistics()

println(matrixSummary.mean)
println(matrixSummary.min)
println(matrixSummary.max)
println(matrixSummary.variance)
println(matrixSummary.numNonzeros)

- 평균( mean )과 분산( variance )을 보면, 두번째 특성치( feature )가 다른 특성치보다 매우 높게 나타나는것을 확인할 수 있으며, 이 특성치가 모델에서 많은 영향을 주고 있다고 판단할 수 있음.
- 모든 특성치가 모델에 동일한 수준으로 영향을 주도록 하기 위해서, 평균이 0이고, 분산 1인  표준정규분포로 변환하는것이 필요
![standardize](Classification_21.jpg "standardize")

- Spark의 StandardScaler 클래스를 이용해서 vector들을 표준화함.

In [ ]:
import org.apache.spark.mllib.feature.StandardScaler
val scaler = new StandardScaler(withMean = true, withStd = true).fit(vectors)
val scaledData = data.map(lp => LabeledPoint(lp.label, scaler.transform(lp.features)))

println(data.first.features)
println(scaledData.first.features)

println((0.789131 - 0.41225805299526636)/ math.sqrt(0.1097424416755897))

In [ ]:
val lrModelScaled = LogisticRegressionWithSGD.train(scaledData, numIterations)
val lrTotalCorrectScaled = scaledData.map { point => 
    if (lrModelScaled.predict(point.features) == point.label) 1 else 0
}.sum

val lrAccuracyScaled = lrTotalCorrectScaled / numData
val lrPredictionsVsTrue = scaledData.map { point =>
    (lrModelScaled.predict(point.features), point.label)
}

val lrMetricsScaled = new BinaryClassificationMetrics(lrPredictionsVsTrue)
val lrPr = lrMetricsScaled.areaUnderPR
val lrRoc = lrMetricsScaled.areaUnderROC

println(f"${lrModelScaled.getClass.getSimpleName}\nAccuracy: ${lrAccuracyScaled * 100}%2.4f%%\nArea under PR: ${lrPr * 100.0}%2.4f%%\nArea under ROC: ${lrRoc * 100.0}%2.4f%%")

## 6.2. Additional features 

- 계산의 편리성을 위해서 카테고리 변수와 문자열 변수는 사용하지 않았음.
- 카테고리 변수를 추가해보자.

In [ ]:
val categories = records.map(r => r(3)).distinct.collect.zipWithIndex.toMap
val numCategories = categories.size

println(categories)
println(numCategories)

In [ ]:
val dataCategories = records.map { r =>
    val trimmed = r.map(_.replaceAll("\"", ""))
    val label = trimmed(r.size - 1).toInt
    val categoryIdx = categories(r(3))
    val categoryFeatures = Array.ofDim[Double](numCategories)
    categoryFeatures(categoryIdx) = 1.0
    val otherFeatures = trimmed.slice(4, r.size - 1).map(d => if (d == "?") 0.0 else d.toDouble)
    val features = categoryFeatures ++ otherFeatures
    LabeledPoint(label, Vectors.dense(features))
}
println(dataCategories.first)

// 표준화 
val scalerCats = new StandardScaler(withMean = true, withStd = true).fit(dataCategories.map(lp => lp.features))
val scaledDataCats = dataCategories.map(lp => LabeledPoint(lp.label, scalerCats.transform(lp.features)))

println(dataCategories.first.features)
println(scaledDataCats.first.features)

- 카테고리 변수를 추가한 데이터로 예측모델 학습 및 평가

In [ ]:
val lrModelScaledCats = LogisticRegressionWithSGD.train(scaledDataCats, numIterations)
val lrTotalCorrectScaledCats = scaledDataCats.map { point =>
    if (lrModelScaledCats.predict(point.features) == point.label) 1 else 0
}.sum

val lrAccuracyScaledCats = lrTotalCorrectScaledCats / numData
val lrPredictionsVsTrueCats = scaledDataCats.map { point => 
    (lrModelScaledCats.predict(point.features), point.label)
}

val lrMetricsScaledCats = new BinaryClassificationMetrics(lrPredictionsVsTrueCats)
val lrPrCats = lrMetricsScaledCats.areaUnderPR
val lrRocCats = lrMetricsScaledCats.areaUnderROC

println(f"${lrModelScaledCats.getClass.getSimpleName}\nAccuracy:${lrAccuracyScaledCats * 100}%2.4f%%\nArea under PR: ${lrPrCats *100.0}%2.4f%%\nArea under ROC: ${lrRocCats * 100.0}%2.4f%%")

## 6.3. Using the correct form of data

- 각각의 학습모델마다 맞는 올바른 데이터 형식을 사용이 성능에 영향을 줌
- 나이브베이즈 모델에 맞게 데이터 변환 작업

In [ ]:
val dataNB = records.map { r =>
    val trimmed = r.map(_.replaceAll("\"", ""))
    val label = trimmed(r.size - 1).toInt
    val categoryIdx = categories(r(3))
    val categoryFeatures = Array.ofDim[Double](numCategories)
    categoryFeatures(categoryIdx) = 1.0
    LabeledPoint(label, Vectors.dense(categoryFeatures))
}
println(dataNB.first)

val nbModelCats = NaiveBayes.train(dataNB)
val nbTotalCorrectCats = dataNB.map { point =>
    if (nbModelCats.predict(point.features) == point.label) 1 else 0
}.sum

val nbAccuracyCats = nbTotalCorrectCats / numData
val nbPredictionsVsTrueCats = dataNB.map { point =>
    (nbModelCats.predict(point.features), point.label)
}

val nbMetricsCats = new BinaryClassificationMetrics(nbPredictionsVsTrueCats)
val nbPrCats = nbMetricsCats.areaUnderPR
val nbRocCats = nbMetricsCats.areaUnderROC
println(f"${nbModelCats.getClass.getSimpleName}\nAccuracy: ${nbAccuracyCats * 100}%2.4f%%\nArea under PR: ${nbPrCats * 100.0}%2.4f%%\nArea under ROC: ${nbRocCats * 100.0}%2.4f%%")


## 6.4. Tuning model parameters

### 6.4.1. Linear models

- logistic regression과 SVM은 파라메터가 동일
- stochastic gradient descent (SGD)의 최적화 기술을 동일함.
- 차이점은 다른 loss function 를 갖음

In [ ]:
class LogisticRegressionWithSGD private (
   private var stepSize: Double,
   private var numIterations: Int, 
   private var regParam: Double,
   private var miniBatchFraction: Double)
   extends GeneralizedLinearAlgorithm[LogisticRegressionModel]

- SparkML의 logistic regression에 대한 생성자 정의는 위와 같음.
- stepSize, numIterations, regParam, and miniBatchFraction는 필수
- regParam 생략 가능

In [ ]:
private val gradient = new LogisticGradient()
private val updater = new SimpleUpdater()
override val optimizer = new GradientDescent(gradient, updater).setStepSize(stepSize).setNumIterations(numIterations).setRegParam(regParam).setMiniBatchFraction(miniBatchFraction)

- 위의 코드는 Gradient, Updater, and Optimizer을 초기설정하는 방법

In [ ]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.optimization.Updater
import org.apache.spark.mllib.optimization.SimpleUpdater
import org.apache.spark.mllib.optimization.L1Updater
import org.apache.spark.mllib.optimization.SquaredL2Updater
import org.apache.spark.mllib.classification.ClassificationModel

def trainWithParams(input: RDD[LabeledPoint], regParam: Double, numIterations: Int, updater: Updater, stepSize: Double) = {
    val lr = new LogisticRegressionWithSGD
    lr.optimizer.setNumIterations(numIterations).setUpdater(updater).setRegParam(regParam).setStepSize(stepSize)
    lr.run(input)
}

def createMetrics(label: String, data: RDD[LabeledPoint], model: ClassificationModel) = {
   val scoreAndLabels = data.map { point =>
       (model.predict(point.features), point.label)
   }
   val metrics = new BinaryClassificationMetrics(scoreAndLabels)
   (label, metrics.areaUnderROC)
}

scaledDataCats.cache

- Iterations 변경에 따른 성능 영향 확인

In [ ]:
val iterResults = Seq(1, 5, 10, 50).map { param =>
    val model = trainWithParams(scaledDataCats, 0.0, param, new SimpleUpdater, 1.0)
    createMetrics(s"$param iterations", scaledDataCats, model)
}

iterResults.foreach { 
    case (param, auc) => println(f"$param, AUC =${auc * 100}%2.2f%%") 
}

- Step size변경에 따른 성능 영향 확인

In [ ]:
val stepResults = Seq(0.001, 0.01, 0.1, 1.0, 10.0).map { param =>
    val model = trainWithParams(scaledDataCats, 0.0, numIterations, new SimpleUpdater, param)
    createMetrics(s"$param step size", scaledDataCats, model)
}

stepResults.foreach { 
    case (param, auc) => println(f"$param, AUC = ${auc * 100}%2.2f%%") 
}

- Regularization( 정규화 ) 변경에 따른 성능 영향 확인
- updater 클래스는 SparkML에서 구현한 Regularization 모듈
- Regularization는 많은 변수를 갖는 복잡한 모델에 효과적으로 패널티를 주므로 해서 학습할때 과학습( over-fitting )을 피하는데 도움을 줌.
- 변수가 많을때( feature dimension is very high )에 중요한 요소임.
- regularization의 종류
- 1) SimpleUpdater: no regularization
- 2) SquaredL2Updater: the squared L2-norm of the weight vector
- 3) L1Updater: L1-norm of the weight vector

In [ ]:
val regResults = Seq(0.001, 0.01, 0.1, 1.0, 10.0).map { param =>
    val model = trainWithParams(scaledDataCats, param, numIterations, new SquaredL2Updater, 1.0)
    createMetrics(s"$param L2 regularization parameter", scaledDataCats, model)
}

regResults.foreach { 
    case (param, auc) => println(f"$param, AUC =${auc * 100}%2.2f%%") 
}

## 6.4.2. Decision trees

- 나무의 깊이(tree depth) 와 불순도( impurity )를 파라마터로 설정
- 나무의 깊이가 깊을수록 학습데이터에 대해서는 성능이 좋아지만,, 과학습( over-fitting )이 되어 예측력은 떨어짐.

In [ ]:
import org.apache.spark.mllib.tree.impurity.Impurity
import org.apache.spark.mllib.tree.impurity.Entropy
import org.apache.spark.mllib.tree.impurity.Gini
def trainDTWithParams(input: RDD[LabeledPoint], maxDepth: Int,impurity: Impurity) = {
    DecisionTree.train(input, Algo.Classification, impurity,maxDepth)
}

val dtResultsEntropy = Seq(1, 2, 3, 4, 5, 10, 20).map { param =>
    val model = trainDTWithParams(data, param, Entropy)
    val scoreAndLabels = data.map { point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
                                                       
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (s"$param tree depth", metrics.areaUnderROC)
}

dtResultsEntropy.foreach { 
    case (param, auc) => println(f"$param,AUC = ${auc * 100}%2.2f%%") 
}


### 6.4.3. The naïve Bayes model

- 나이브베이즈 모델의 파라메타는 lambda 임.
- lambda는 데이터셋에서 분류와 변수가 같이 일어나지 않는 상황을 처리하기 위해서 추가적인 smoothing을 컨트롤 함.
- This parameter controls additive smoothing, which handles the case when a class and feature value do not occur together in the dataset.

In [ ]:
def trainNBWithParams(input: RDD[LabeledPoint], lambda: Double) = {
    val nb = new NaiveBayes
    nb.setLambda(lambda)
    nb.run(input)
}

val nbResults = Seq(0.001, 0.01, 0.1, 1.0, 10.0).map { param =>
    val model = trainNBWithParams(dataNB, param)
    val scoreAndLabels = dataNB.map { point =>
        (model.predict(point.features), point.label)
    }                                                      
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (s"$param lambda", metrics.areaUnderROC)
}

nbResults.foreach { 
    case (param, auc) => println(f"$param, AUC =${auc * 100}%2.2f%%")
}

## 6.5. Cross-validation

In [ ]:
val trainTestSplit = scaledDataCats.randomSplit(Array(0.6, 0.4), 121)
val train = trainTestSplit(0)
val test = trainTestSplit(1)

val regResultsTest = Seq(0.0, 0.001, 0.0025, 0.005, 0.01).map { param =>
    val model = trainWithParams(train, param, numIterations, new SquaredL2Updater, 1.0)
    createMetrics(s"$param L2 regularization parameter", test, model)
}

regResultsTest.foreach { 
    case (param, auc) => println(f"$param, AUC = ${auc * 100}%2.6f%%")
}

- 나무모형에서 cross-validation시 예측력의 변화 확인

In [ ]:
val dtResultsEntropy = Seq(1, 2, 3, 4, 5, 10, 20).map { param =>
    val model = trainDTWithParams(train, param, Entropy)
    val scoreAndLabels = test.map { point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
                                                       
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (s"$param tree depth", metrics.areaUnderROC)
}

dtResultsEntropy.foreach { 
    case (param, auc) => println(f"$param,AUC = ${auc * 100}%2.2f%%") 
}